In [1]:
import pandas as pd
from nltk import word_tokenize
from tqdm import tqdm
tqdm.pandas()

from sklearn.utils import shuffle

In [2]:
path_windows = "D:/Sync/Thesis/Datasources/Preprocessed/Combined/Taxonomy/Normal/Annotated/combined_annotations.csv"
path_mac = '/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/Combined/Taxonomy/Normal/Annotated/combined_annotations.csv'
path_conference = '/Users/ivowings/Sync/Thesis/Datasources/Conference data/conference_data.csv'

In [3]:
annotations = pd.read_csv(path_mac,sep=';')
annotations = shuffle(annotations,random_state=123)
annotations = annotations.rename(columns={'recovered_gram':'candidate_skill'})
annotations.head()

,left_context,candidate_skill,right_context,label
19073,assist their development enable them to become...,independent,and in turn become business mentor themselves,0
13367,excellent,understanding,of the web stack,0
19817,member handbook and offer service of a,polite,efficient and professional manner,1
9569,make them easy to use,and understand,and working with ux and,0
5291,the uncertain waters exceptional communication,skills verbal and,written highly organised,0


In [4]:
candidate_skill_split = annotations['candidate_skill'].str.split(' ',expand=True)
candidate_skill_split = candidate_skill_split.add_prefix('token_')
candidate_skill_split.head()

,token_0,token_1,token_2,token_3
19073,independent,None,None,None
13367,understanding,None,None,None
19817,polite,None,None,None
9569,and,understand,None,None
5291,skills,verbal,and,None


In [5]:
annotations = candidate_skill_split.join(annotations)
annotations.head()

,token_0,token_1,token_2,token_3,left_context,candidate_skill,right_context,label
19073,independent,None,None,None,assist their development enable them to become...,independent,and in turn become business mentor themselves,0
13367,understanding,None,None,None,excellent,understanding,of the web stack,0
19817,polite,None,None,None,member handbook and offer service of a,polite,efficient and professional manner,1
9569,and,understand,None,None,make them easy to use,and understand,and working with ux and,0
5291,skills,verbal,and,None,the uncertain waters exceptional communication,skills verbal and,written highly organised,0


In [6]:
annotations['token_0'] = annotations['token_0']+':'+annotations['label'].astype(str)
annotations['token_1'] = annotations['token_1']+':'+annotations['label'].astype(str)
annotations['token_2'] = annotations['token_2']+':'+annotations['label'].astype(str)
annotations['token_3'] = annotations['token_3']+':'+annotations['label'].astype(str)
annotations.head()

,token_0,token_1,token_2,token_3,left_context,candidate_skill,right_context,label
19073,independent:0,NaN,NaN,NaN,assist their development enable them to become...,independent,and in turn become business mentor themselves,0
13367,understanding:0,NaN,NaN,NaN,excellent,understanding,of the web stack,0
19817,polite:1,NaN,NaN,NaN,member handbook and offer service of a,polite,efficient and professional manner,1
9569,and:0,understand:0,NaN,NaN,make them easy to use,and understand,and working with ux and,0
5291,skills:0,verbal:0,and:0,NaN,the uncertain waters exceptional communication,skills verbal and,written highly organised,0


In [7]:
annotations = annotations.fillna('')
annotations['combined'] = annotations['left_context'] + ' ' + annotations['token_0'] + ' ' + annotations['token_1'] + ' '+ annotations['token_2'] + ' ' + annotations['token_2'] + ' '+ annotations['right_context']+'.'
annotations.head()

,token_0,token_1,token_2,token_3,left_context,candidate_skill,right_context,label,combined
19073,independent:0,,,,assist their development enable them to become...,independent,and in turn become business mentor themselves,0,assist their development enable them to become...
13367,understanding:0,,,,excellent,understanding,of the web stack,0,excellent understanding:0 of the web stack.
19817,polite:1,,,,member handbook and offer service of a,polite,efficient and professional manner,1,member handbook and offer service of a polite...
9569,and:0,understand:0,,,make them easy to use,and understand,and working with ux and,0,make them easy to use and:0 understand:0 and...
5291,skills:0,verbal:0,and:0,,the uncertain waters exceptional communication,skills verbal and,written highly organised,0,the uncertain waters exceptional communication...


In [8]:
annotations.shape[0]

20836

In [58]:
train = annotations[['combined']].iloc[0:round(0.8*len(annotations)),]
val = annotations[['combined']].drop(train.index).iloc[0:round(0.1*len(annotations)),]
test = annotations[['combined']].drop(train.index).drop(val.index).iloc[0:round(0.1*len(annotations)),]


In [59]:
test = annotations[['combined']].sample(frac=0.2,random_state=123)
train = annotations.drop(test.index)
train = train[['combined']]

In [60]:
#Breaking each sentence down into tokens for annotation
token_df = pd.DataFrame()

dataset= test

for index in tqdm(range(0,len(dataset['combined']))):
    tokens = pd.DataFrame(word_tokenize(dataset['combined'].iloc[index]),columns=['tokens'])
    tokens['index'] = index
    
    token_df = pd.concat([token_df,tokens])

100%|██████████| 2083/2083 [00:03<00:00, 546.38it/s]


In [61]:
annotationcolumn = token_df['tokens'].str.split(':',expand=True)
annotationcolumn.head()

,0,1
0,variety,None
1,of,None
2,diverse,0
3,tasks,None
4,with,None


In [62]:
token_df['annotation'] = annotationcolumn[1]
token_df['tokens'] = annotationcolumn[0]
token_df.head()

,tokens,index,annotation
0,variety,0,None
1,of,0,None
2,diverse,0,0
3,tasks,0,None
4,with,0,None


In [64]:
token_df['annotation'] = token_df['annotation'].fillna('O')
token_df.annotation[token_df.annotation=='0'] = 'O'
token_df.annotation[token_df.annotation=='1'] = 'Soft_Skill'
token_df.annotation[token_df.annotation=='2'] = 'Hard_Skill'
token_df.head()

<ipython-input-64-a4c4080af63a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  token_df.annotation[token_df.annotation=='0'] = 'O'
<ipython-input-64-a4c4080af63a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  token_df.annotation[token_df.annotation=='1'] = 'Soft_Skill'
<ipython-input-64-a4c4080af63a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  token_df.annotation[token_df.annotation=='2'] = 'Hard_Skill'


,tokens,index,annotation
0,variety,0,O
1,of,0,O
2,diverse,0,O
3,tasks,0,O
4,with,0,O


In [65]:
token_df = token_df.rename(columns={'index':'sentence'})
token_df = token_df[['tokens','annotation']]
token_df.head()

,tokens,annotation
0,variety,O
1,of,O
2,diverse,O
3,tasks,O
4,with,O


In [66]:
windows_path_spacy = "C:/Users/Ivo/Desktop/spacy.tsv"
windows_path_spacy_json = "C:/Users/Ivo/Desktop/spacy.json"

mac_path_spacy = '/Users/ivowings/Desktop/spacy.tsv'
mac_path_spacy_json = '/Users/ivowings/Desktop/spacy.json'

token_df.to_csv(mac_path_spacy,sep='\t', index=False,header=False)

In [67]:
#Converting the pandas dataframe to spacy 2 format
import json
import logging
import sys

test = list()

def tsv_to_json_format(input_path,output_path,unknown_label):
    try:
        f=open(input_path,'r') # input file
        fp=open(output_path, 'w') # output file
        data_dict={}
        annotations =[]
        label_dict={}
        s=''
        start=0
        for line in f:
            if line[0:len(line)-1]!='.\tO':
                word,entity=line.split('\t')
                s+=word+" "
                entity=entity[:len(entity)-1]
                if entity!=unknown_label:
                    if len(entity) != 1:
                        d={}
                        d['text']=word
                        d['start']=start
                        d['end']=start+len(word)-1  
                        try:
                            label_dict[entity].append(d)
                        except:
                            label_dict[entity]=[]
                            label_dict[entity].append(d) 
                start+=len(word)+1
            else:
                data_dict['content']=s
                s=''
                label_list=[]
                for ents in list(label_dict.keys()):
                    for i in range(len(label_dict[ents])):
                        if(label_dict[ents][i]['text']!=''):
                            l=[ents,label_dict[ents][i]]
                            for j in range(i+1,len(label_dict[ents])): 
                                if(label_dict[ents][i]['text']==label_dict[ents][j]['text']):  
                                    di={}
                                    di['start']=label_dict[ents][j]['start']
                                    di['end']=label_dict[ents][j]['end']
                                    di['text']=label_dict[ents][i]['text']
                                    l.append(di)
                                    label_dict[ents][j]['text']=''
                            label_list.append(l)                          
                            
                for entities in label_list:
                    label={}
                    label['label']=[entities[0]]
                    label['points']=entities[1:]
                    annotations.append(label)
                data_dict['annotation']=annotations
                annotations=[]
                json.dump(data_dict, fp)
                fp.write('\n')
                data_dict={}
                start=0
                label_dict={}
    except Exception as e:
        logging.exception("Unable to process file" + "\n" + "error = " + str(e))
        return None

tsv_to_json_format(mac_path_spacy,mac_path_spacy_json,'')

In [68]:
# Convert json file to spaCy format.
import logging
import argparse
import sys
import os
import json
import pickle


def main(input_path):
    try:
        training_data = []
        lines=[]
        with open(input_path, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                point = annotation['points'][0]
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

        return training_data
        
    except Exception as e:
        logging.exception("Unable to process " + input_path + "\n" + "error = " + str(e))
        return None


In [69]:
training_data = main(mac_path_spacy_json)

[('variety of diverse tasks with ', {'entities': []}),
 ('all member of the multidisciplinary team work collaboratively to ass and meet ',
  {'entities': [(41, 45, 'Soft_Skill'), (46, 61, 'Soft_Skill')]}),
 ('execute cost effective ', {'entities': []}),
 ('location renfrewshire employer independent hotel operation job type ',
  {'entities': []}),
 ('and clearly communicating details to all ', {'entities': []}),
 ('the firm finance function that will report directly to a dynamic and inspirational finance director ',
  {'entities': []}),
 ('we be look for a candidate who thrive within a dynamic environment but who can pay close attention to detail and ',
  {'entities': []}),
 ('talented new business focused sales person to play an instrumental role ',
  {'entities': []}),
 ('bonus in touch recruitment be work on behalf of an dynamic care provider to recruit an experience care home manager for ',
  {'entities': []}),
 ('at whizzkidz one of the uk s most dynamic charity ', {'entities': []}

In [70]:
# #for generating confusion matrices
# with open('json_data_spacy_confusion_matrices.json', 'w') as f:
#     json.dump(training_data, f)

In [22]:
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

for text, annot in tqdm(training_data): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db

100%|██████████| 20836/20836 [00:10<00:00, 2061.28it/s]
